# Exercise 8

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

FS = 15 # Fontsize of caption

## 8.1 Grayscale edge detector

In [ ]:
# Load the image in grayscale
I = cv2.imread('bike.png', cv2.IMREAD_GRAYSCALE)

# Normalize the image to 0-1
I = I.astype(np.float32) / 255.0

# Define the structuring element (rectangle with size 4x5)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 4))

# Perform dilation on the original image
I_dilated = cv2.dilate(I, kernel)

# Perform edge detection by subtracting the original image from the dilated image
edge = I_dilated - I

# Threshold to convert to binary image
BW = edge > 0.2  # 0.2 is the threshold value

# Show images
plt.figure(figsize = (10, 10))

plt.subplot(2, 2, 1)
plt.imshow(I, cmap = 'gray')
plt.title('Original Image', fontsize = FS)

plt.subplot(2, 2, 2)
plt.imshow(I_dilated, cmap = 'gray')
plt.title('Dilated Image', fontsize = FS)

plt.subplot(2, 2, 3)
plt.imshow(edge, cmap = 'gray')
plt.title('Edge = Dilation - Origin', fontsize = FS)

plt.subplot(2, 2, 4)
plt.imshow(BW, cmap = 'gray')
plt.title('BW of Edge', fontsize = FS)

plt.tight_layout()
plt.show()
